In [1]:
import cv2

In [25]:
img = cv2.imread('./data_processed/div/9UN_456_em_733.inkml.jpg')

In [26]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255,
	cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [27]:
output = cv2.connectedComponentsWithStats(
	thresh, 8, cv2.CV_32S)
(numLabels, labels, stats, centroids) = output

In [28]:
stats

array([[     0,      0,    515,    338, 170462],
       [   267,     22,     37,     50,    355],
       [    30,    121,    457,     51,   2636],
       [   297,    224,     56,     93,    617]], dtype=int32)

In [6]:
## TODO: Make this not crash when you close the windows
# for i in range(0, numLabels):
#     # if this is the first component then we examine the
#     # *background* (typically we would just ignore this
#     # component in our loop)
#     if i == 0:
#         text = "examining component {}/{} (background)".format(
#             i + 1, numLabels)
#     # otherwise, we are examining an actual connected component
#     else:
#         text = "examining component {}/{}".format( i + 1, numLabels)
#     # print a status message update for the current connected
#     # component
#     print("[INFO] {}".format(text))
#     # extract the connected component statistics and centroid for
#     # the current label
#     x = stats[i, cv2.CC_STAT_LEFT]
#     y = stats[i, cv2.CC_STAT_TOP]
#     w = stats[i, cv2.CC_STAT_WIDTH]
#     h = stats[i, cv2.CC_STAT_HEIGHT]
#     area = stats[i, cv2.CC_STAT_AREA]
#     (cX, cY) = centroids[i]

#     output = img.copy()
#     cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 3)
#     cv2.circle(output, (int(cX), int(cY)), 4, (0, 0, 255), -1)
#     componentMask = (labels == i).astype("uint8") * 255
#     # show our output image and connected component mask
#     cv2.imshow("Output", output)
#     cv2.imshow("Connected Component", componentMask)
#     cv2.waitKey(0)

In [29]:
new_stats = []
added = [False for i in range(len(stats))]
threshhold = .5
for i in range(1,len(stats)):
    if not added[i]:
        (xi, yi, wi, hi, ai) = stats[i]
        # print((xi, yi, wi, hi, ai))
        for j  in range(i+1,len(stats)):
            if not added[j]:
                (xj, yj, wj, hj, aj) = stats[j]
                # print('\t',(xj, yj, wj, hj, aj))
                if xi<=xj:
                    if xi+wi>=xj+wj:
                        # print((xi, yi, wi, hi, ai),(xj, yj, wj, hj, aj))
                        hi = max(yi+hi,yj+hj)
                        yi = min(yi,yj)
                        hi = hi-yi
                        ai += aj
                        added[j] = True
                        # print((xi, yi, wi, hi, ai))
                    elif xi+wi>=xj:
                        o = xi+wi-xj
                        if o/wi>threshhold or o/wj>threshhold:
                            # print((xi, yi, wi, hi, ai),(xj, yj, wj, hj, aj))
                            wi = xj+wj-xi
                            hi = max(yi+hi,yj+hj)
                            yi = min(yi,yj)
                            hi = hi-yi
                            ai += aj    
                            added[j] = True
                            # print((xi, yi, wi, hi, ai))
                if xj<xi:
                    if xj+wj>=xi+wi:
                        # print((xi, yi, wi, hi, ai),(xj, yj, wj, hj, aj))
                        xi = xj
                        wi = wj
                        hi = max(yi+hi,yj+hj)
                        yi = min(yi,yj)
                        hi = hi-yi
                        ai += aj
                        added[j] = True
                        # print((xi, yi, wi, hi, ai))
                    elif xj+wj>=xi:
                        o = xj+wj-xi
                        if o/wi>threshhold or o/wj>threshhold:
                            # print((xi, yi, wi, hi, ai),(xj, yj, wj, hj, aj))
                            wi = xj+wj-xi
                            hi = max(yi+hi,yj+hj)
                            yi = min(yi,yj)
                            hi = hi-yi
                            ai += aj
                            added[j] = True
                            # print((xi, yi, wi, hi, ai))
        new_stats.append([xi, yi, wi, hi, ai])                
        added[i] = True

(267, 22, 37, 50, 355) (30, 121, 457, 51, 2636)
(30, 22, 457, 150, 2991)
(30, 22, 457, 150, 2991) (297, 224, 56, 93, 617)
(30, 22, 457, 295, 3608)


In [30]:
for i in range(len(new_stats)):
    
    x = new_stats[i][0]
    y = new_stats[i][1]
    w = new_stats[i][2]
    h = new_stats[i][3]
    area = new_stats[i][4]
    # (cX, cY) = centroids[i]

    output = img.copy()
    cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 3)
    # cv2.circle(output, (int(cX), int(cY)), 4, (0, 0, 255), -1)
    # componentMask = (labels == i).astype("uint8") * 255
    # show our output image and connected component mask
    cv2.imshow("Output", output)
    # cv2.imshow("Connected Component", componentMask)
    cv2.waitKey(0)